<a href="https://colab.research.google.com/github/BurakKizilkaya/Machine-Learning/blob/master/KIZILKAYA_forest_fires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNG 562 - HOMEWORK 1

# Question  : Forest Fires


In [0]:
#upload data(.csv file)
from google.colab import files
uploaded = files.upload()


In [0]:
#import data
import pandas as pd
data = pd.read_csv("forestfires.csv")

data

In [0]:
#creating dummy variables for categorical columns
data = pd.get_dummies(data, columns=['month', 'day'], drop_first=True)

data.head()

In [0]:
data.columns

In [0]:
#making small fires(less than 100 m2) 0 others 1 according to http://www3.dsi.uminho.pt/pcortez/fires.pdf)
data[data['area'] > 0 ] = 1

In [0]:
data.columns, data.shape

In [0]:
#normalizing data
import numpy as np

tempx = np.array(data.drop('area', axis=1))
tempy = np.array(data['area'])
x = (tempx - tempx.min(0)) / tempx.ptp(0)
y = (tempy - tempy.min(0)) / tempy.ptp(0)

print(x)
print(y)



In [0]:
x.min(), x.max(), x.mean(axis=0), x.shape

In [0]:
#split test and train data
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=0)


## Linear Regression and Cross Validation



In [0]:
#linear regression with sklearn
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_val_predict
import matplotlib.pyplot as plot


linreg = LinearRegression().fit(xtrain, ytrain)
yscore = linreg.score(xtest,ytest)

print(linreg.score(xtrain, ytrain))
print(yscore)




In [0]:
#cross validation
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_val_predict
lm = LinearRegression()

scores = cross_val_score(lm, xtrain, ytrain, cv = 5)    #cv is the number of folds, scores will give an array of scores

print(scores)
print(np.mean(scores)) 
print(np.std(scores))


## Decision Tree and Grid Search

In [0]:
#decision tree with sklearn 
from sklearn.tree import DecisionTreeClassifier  
 
dt = DecisionTreeClassifier()   
dt.fit(xtrain, ytrain) 

ypred = dt.predict(xtest)

print("Accuracy:",metrics.accuracy_score(ytest, ypred))
print("Precision:",metrics.precision_score(ytest, ypred))
print("Recall:",metrics.recall_score(ytest, ypred))

In [0]:
#grid search for decision tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
tuned_parameters = [{'criterion': ['gini', 'entropy'], 'max_features' : ['auto', 'log2', 1.0, 8, 0.66], 'min_samples_leaf' : [1, 2, 3]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(DecisionTreeClassifier(), tuned_parameters, cv=5, scoring='%s_macro' % score)
    clf.fit(xtrain, ytrain)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    ytrue, ypred = ytest, clf.predict(xtest)
    print(classification_report(ytrue, ypred))
    print()

## K-Nearest Neighbour and Grid Search

In [0]:
#knn with sklearn
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier()
neigh.fit(xtrain, ytrain) 
ypred = neigh.predict(xtest)

print("Accuracy:",metrics.accuracy_score(ytest, ypred))
print("Precision:",metrics.precision_score(ytest, ypred))
print("Recall:",metrics.recall_score(ytest, ypred))

In [0]:
#grid search for support vector machine
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import svm

# Set the parameters by cross-validation
tuned_parameters = [{'n_neighbors': [3, 4, 5, 6, 7, 8, 9], 'weights' : ['uniform', 'distance']}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(KNeighborsClassifier(), tuned_parameters, cv=5, scoring='%s_macro' % score)
    clf.fit(xtrain, ytrain)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    ytrue, ypred = ytest, clf.predict(xtest)
    print(classification_report(ytrue, ypred))
    print()